# 그룹별 Tension 예측을 위한 데이터 전처리 코드

In [1]:
print('start')
import pandas as pd
import numpy as np 
import os 
import re 

start


In [2]:
import datetime
def group_sec(data , freq):
    
    data['datetime'] =  pd.to_datetime(data['timestamp'], unit='s')
    
                
    #data['datetime'] = pd.to_timedelta(data['timestamp'], unit='s')
    df = data.set_index('datetime')
    
    ten_sec_df = df.groupby(pd.Grouper(freq=freq)).agg('max')
    ten_sec_df = ten_sec_df.interpolate()

    ten_sec_df = ten_sec_df.reset_index()
    return ten_sec_df

In [3]:
pre_pre_dir = '/workspace/data/dataset_2019/'
for d in os.listdir(pre_pre_dir):

    pre_dir = pre_pre_dir + d +"/"
    lst_timestamp = os.listdir(pre_dir)
    print(d)

    for ts in lst_timestamp:
        time_pre_dir = pre_dir + ts + '/'
        for ma in os.listdir(time_pre_dir):
            if ma[-4] == '.':
                continue
            t = time_pre_dir+ma+'/'
            df = pd.DataFrame()
            for detail_timestamp in os.listdir(t):
                file_dir = t + detail_timestamp

                tmp_data = pd.read_csv(file_dir)
                tmp_data['timestamp'] += int(detail_timestamp.replace('.csv',''))

                tmp_data = group_sec(tmp_data,'5s')
                tmp_data.rename(columns = lambda x: ma +"__"+ x, inplace = True)

                #print(tmp_data.shape)
                df = pd.concat([df,tmp_data],ignore_index=True)
            df.to_csv(time_pre_dir+ma+'.csv')




109
108
118
105
113
116
110
104
115
111
101
114
103
106
119
102
107
117
120
112


In [4]:
import os
import pandas as pd 
import numpy as np

ppre_dir = '/workspace/data/dataset_2019/'

for filename in os.listdir(ppre_dir):
    
    _pre_dir = ppre_dir + filename+'/'
    df = pd.DataFrame()
    print(_pre_dir)
    for d in os.listdir(_pre_dir):
        cnt = 0 
        pre_dir = _pre_dir + d + '/'
        check = False
        for file in os.listdir(pre_dir):
            if  ('.' not in file )or ('label' in file ):
                continue
            #print(file)
            if cnt == 0 :
                cnt +=1 
                t_df = pd.read_csv(pre_dir + file,index_col=0)
                if t_df.shape[0] <= 0 :
                    break
                t_df.rename(columns={ file.replace('.csv','')+'__datetime':'datetime'}, inplace = True)

                t_df['datetime'] = pd.to_datetime(t_df['datetime']).astype(str)
                t_df =t_df.drop(columns = [file.replace('.csv','')+'__timestamp'])
                #e4Acc__timestamp

                continue

            tmp_df = pd.read_csv(pre_dir + file,index_col=0)
            if tmp_df.shape[0] <= 0 :
                break        
            word = file.replace('.csv','')+'__timestamp'


            tmp_df =tmp_df.drop(columns = [file.replace('.csv','')+'__timestamp'])

            tmp_df.rename(columns={ file.replace('.csv','')+'__datetime':'datetime'}, inplace = True)
            tmp_df['datetime'] = pd.to_datetime(tmp_df['datetime']).astype(str)
            t_df = pd.merge(t_df, tmp_df, on=['datetime'], how = 'inner')


        df = pd.concat([df,t_df])
    df = df.set_index(['datetime'])
    df = df.sort_index()
    #ppre_dir ='/workspace/data/user01-06/user02/'

    lst = os.listdir(_pre_dir)
    concat_label = pd.DataFrame()
    for f in lst:
        
        file_path = f'{_pre_dir}{f}/{f}_label.csv'
        #print(file_path)
        label = pd.read_csv(file_path)
        if label.shape[0] <= 0 :
            continue
        label['datetime'] =  pd.to_datetime(label['ts'], unit='s')

        label = label.set_index(['datetime'])
        #print(label.index.duplicated())
        #duplicated_values = label[label.index.duplicated()]
        #print(duplicated_values)
        label = label[~label.index.duplicated()]
        label_5s = label.resample('5s').ffill()

        concat_label = pd.concat([concat_label,label_5s])
    concat_label = concat_label.sort_index()
    concat_label = concat_label.drop(columns = ['ts'])
    df = df.reset_index()
    concat_label = concat_label.reset_index()
    df['datetime'] = pd.to_datetime(df['datetime'])
    concat_label['datetime'] = pd.to_datetime(concat_label['datetime'])
    merged_df = pd.merge(df, concat_label, on='datetime', how='inner')
    merged_df.fillna(0)
    merged_df = merged_df.set_index('datetime')
    merged_df.to_csv(f'processed_data/tmp_merged_{filename}.csv')

/workspace/data/dataset_2019/109/
/workspace/data/dataset_2019/108/
/workspace/data/dataset_2019/118/
/workspace/data/dataset_2019/105/
/workspace/data/dataset_2019/113/
/workspace/data/dataset_2019/116/
/workspace/data/dataset_2019/110/
/workspace/data/dataset_2019/104/
/workspace/data/dataset_2019/115/
/workspace/data/dataset_2019/111/
/workspace/data/dataset_2019/101/
/workspace/data/dataset_2019/114/
/workspace/data/dataset_2019/103/
/workspace/data/dataset_2019/106/
/workspace/data/dataset_2019/119/
/workspace/data/dataset_2019/102/
/workspace/data/dataset_2019/107/
/workspace/data/dataset_2019/117/
/workspace/data/dataset_2019/120/
/workspace/data/dataset_2019/112/


In [5]:
!tar -cvf tmp_processed.tar processed_data

processed_data/
processed_data/tmp_merged_110.csv
processed_data/tmp_merged_109.csv
processed_data/tmp_merged_101.csv
processed_data/tmp_merged_104.csv
processed_data/tmp_merged_102.csv
processed_data/tmp_merged_106.csv
processed_data/tmp_merged_118.csv
processed_data/tmp_merged_117.csv
processed_data/tmp_merged_116.csv
processed_data/tmp_merged_103.csv
processed_data/tmp_merged_113.csv
processed_data/tmp_merged_107.csv
processed_data/tmp_merged_119.csv
processed_data/tmp_merged_114.csv
processed_data/tmp_merged_108.csv
processed_data/tmp_merged_111.csv
processed_data/tmp_merged_105.csv
processed_data/tmp_merged_120.csv
processed_data/tmp_merged_112.csv
processed_data/tmp_merged_115.csv
